In [48]:
import asyncio
import json
import threading
from configparser import ConfigParser
from urllib.parse import urlencode
from urllib.request import urlopen

values = {
    "query-url": [
        "https://ru.wikipedia.org/wiki/Python",
        "https://docs.python.org/3/library/threading.html",
    ],
}
urlencode(values)

'query-url=%5B%27https%3A%2F%2Fru.wikipedia.org%2Fwiki%2FPython%27%2C+%27https%3A%2F%2Fdocs.python.org%2F3%2Flibrary%2Fthreading.html%27%5D'

In [129]:
resp = urlopen(
    "http://localhost:8087",
    data=json.dumps(
        {
            "query-url": [
                "https://ru.wikipedia.org/wiki/Python",
                "https://docs.python.org/3/library/threading.html",
            ],
        }
    ).encode(),
)
json.loads(resp.read())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [119]:
def getter():
    resp = urlopen(
        "http://localhost:8087",
        data=urlencode(values).encode(),
    )
    return json.loads(resp.read())


# response = asyncio.gather(*[getter() for _ in range(11)])

In [125]:
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(getter) for _ in range(10)]
    for future in concurrent.futures.as_completed(futures):
        print(future.result())

{'https://docs.python.org/3/library/threading.html': {'the': 468, 'is': 203, 'a': 179, 'to': 155, 'thread': 123}, 'https://ru.wikipedia.org/wiki/Python': {'python': 344, 'в': 315, 'и': 286, 'с': 147, 'на': 145}}
{'https://docs.python.org/3/library/threading.html': {'the': 468, 'is': 203, 'a': 179, 'to': 155, 'thread': 123}, 'https://ru.wikipedia.org/wiki/Python': {'python': 344, 'в': 315, 'и': 286, 'с': 147, 'на': 145}}
{'https://ru.wikipedia.org/wiki/Python': {'python': 344, 'в': 315, 'и': 286, 'с': 147, 'на': 145}, 'https://docs.python.org/3/library/threading.html': {'the': 468, 'is': 203, 'a': 179, 'to': 155, 'thread': 123}}
{'https://docs.python.org/3/library/threading.html': {'the': 468, 'is': 203, 'a': 179, 'to': 155, 'thread': 123}, 'https://ru.wikipedia.org/wiki/Python': {'python': 344, 'в': 315, 'и': 286, 'с': 147, 'на': 145}}
{'https://docs.python.org/3/library/threading.html': {'the': 468, 'is': 203, 'a': 179, 'to': 155, 'thread': 123}, 'https://ru.wikipedia.org/wiki/Python'

In [107]:
resp = urlopen(
    "http://localhost:8087",
    data=urlencode(values).encode(),
)
json.loads(resp.read())

{'https://ru.wikipedia.org/wiki/Python': {'python': 344,
  'в': 315,
  'и': 286,
  'с': 147,
  'на': 145},
 'https://docs.python.org/3/library/threading.html': {'the': 468,
  'is': 203,
  'a': 179,
  'to': 155,
  'thread': 123}}

In [25]:
config = ConfigParser()
config.read("./serve_config.ini")
config.sections()

['server']

In [27]:
config.get("server", "port")

'8087'

In [ ]:
async def tcp_echo_client(message):
    reader, writer = await asyncio.open_connection("localhost", 8888)

    print(f"Send: {message!r}")
    writer.write(message.encode())
    await writer.drain()

    data = await reader.read(1000)

    # print(f"Received: {data.decode()!r}")
    print("Received:")

    print("Close the connection")
    writer.close()
    await writer.wait_closed()
    print(json.loads(data.decode()))
    return json.loads(data.decode())

In [10]:
for _ in range(10):
    th = threading.Thread(
        target=asyncio.run,
        args=(tcp_echo_client("https://ru.wikipedia.org/wiki/Python"),),
    )
    th.start()

Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Send: 'https://ru.wikipedia.org/wiki/Python'
Received:
Close the connection
{'python': 344, 'в': 313, 'и': 286, 'с': 147, 'на': 144, 'архивировано': 139, 'дата': 129, 'для': 116, 'править': 114}
Received:
Close the connection
{'python': 344, 'в': 313, 'и': 286, 'с': 147, 'на': 144, 'архивировано': 139, 'дата': 129, 'для': 116, 'править': 114}
Received:
Close the connection
{'python': 344, 'в': 313, 'и': 286, 'с': 147, 'на': 144, 'архивировано': 139, 'дата': 129, 'для': 116, 'править': 114}
Received:
Close the connection
{'python': 344, 'в': 313, 'и': 286, 'с': 147, 'на': 144, 'архивирован

In [5]:
await tcp_echo_client("https://ru.wikipedia.org/wiki/Python")

Send: 'https://ru.wikipedia.org/wiki/Python'
Received: '{"python": 344, "\\u0432": 313, "\\u0438": 286, "\\u0441": 147, "\\u043d\\u0430": 144, "\\u0430\\u0440\\u0445\\u0438\\u0432\\u0438\\u0440\\u043e\\u0432\\u0430\\u043d\\u043e": 139, "\\u0434\\u0430\\u0442\\u0430": 129, "\\u0434\\u043b\\u044f": 116, "\\u043f\\u0440\\u0430\\u0432\\u0438\\u0442\\u044c": 114}'
Close the connection


{'python': 344,
 'в': 313,
 'и': 286,
 'с': 147,
 'на': 144,
 'архивировано': 139,
 'дата': 129,
 'для': 116,
 'править': 114}